In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# ---------------------------
# Funzioni utili per l'HGA
# ---------------------------

def read_from_file(file_path):
    """
    Ritorno le righe di un file
    """
    with open(file_path, "r") as file:
        lines = [line for line in file]
    return lines

def parse_fvs_file(file_path):
    """
    Funzione per leggere il file e restituire:
    - dizionario dei pesi dei nodi
    - lista degli archi
    """
    lines = read_from_file(file_path)

    node_weights = {}
    adjacency_lines = []
    reading_node_weights = False
    reading_adjacency_matrix = False

    for line in lines:
        line = line.strip()

        #capiamo in che sezione del file ci troviamo
        if line == "NODE_WEIGHT_SECTION":
            reading_node_weights = True
            continue
        elif line == "ADIACENT_LOWER_TRIANGULAR_MATRIX":
            reading_node_weights = False
            reading_adjacency_matrix = True
            continue
        elif line == "EOF":
            break

        # Lettura dei pesi dei nodi
        if reading_node_weights and line:
            parts = line.split()
            if len(parts) >= 2:
                node, weight = int(parts[0]), int(parts[1])
                node_weights[node] = weight

        # Lettura righe della matrice di adiacenza
        if reading_adjacency_matrix and line:
            adjacency_lines.append(list(map(int, line.split())))

    # Conversione della matrice triangolare in lista di archi
    edges = []
    for i, row in enumerate(adjacency_lines, start=1):
        for j, val in enumerate(row, start=1):
            if val == 1:
                edges.append((i, j))

    return node_weights, edges

def is_acyclic(graph, solution):
    """
    Controlla se un grafo è aciclico:
    prende la soluzione inviata e crea un sottografo senza i nodi selezionati
    dopodichè controlla se quel sottografo è aciclico
    """
    nodes_to_remove = [node for node, selected in enumerate(solution, start=1) if selected == 1]
    subgraph = graph.subgraph(set(graph.nodes) - set(nodes_to_remove))
    return nx.is_forest(subgraph)

def fitness(solution, node_weights, graph):
    """
    Calcola la fitness di una soluzione.
    evaluation_count segna quante volte viene chiamata questa funzione
    prende la soluzione inviata e crea un sottografo senza i nodi selezionati
    ritorna la somma dei pesi della soluzione
    se il grafo non è aciclico tornerà la fitness con una penitenza calibrata in base al numero di cicli
    """
    global evaluation_count
    evaluation_count += 1

    nodes_to_remove = [node for node, selected in enumerate(solution, start=1) if selected == 1]
    subgraph = graph.subgraph(set(graph.nodes) - set(nodes_to_remove))
    
    num_cycles = len(nx.cycle_basis(subgraph))
    return sum(node_weights[i + 1] for i, selected in enumerate(solution) if selected == 1) + 10000 * num_cycles

def initialize_population(pop_size, num_nodes, graph):
    """
    Crea una popolazione iniziale di soluzioni random.
    la popolazione è composta da vettori binari lunghi quanto il numero dei nodi del grafo
    su ogni vettore creato chiamiamo una local_search in modo da ottimizzarlo subito
    """
    population = []
    for _ in range(pop_size):
        solution = [1 if np.random.rand() < 0.5 else 0 for _ in range(num_nodes)]
        solution = local_search(solution, graph)
        population.append(solution)
    return population

def tournament_selection(population, fitnesses, k=5):
    """
    Selezione k-tournament usando NumPy.
    Parametri:
    - population (list): Lista di soluzioni candidate.
    - fitnesses (list): Lista dei valori di fitness corrispondenti alle soluzioni.
    - k (int): Numero di partecipanti al torneo.
    Ritorna:
    - list: La soluzione vincente (migliore tra le k selezionate).
    """
    indices = np.random.choice(len(population), size=k, replace=False)
    selected = [(population[i], fitnesses[i]) for i in indices]
    return min(selected, key=lambda x: x[1])[0]

def crossover(parent1, parent2):
    """
    Crossover a tre punti.
    prende 3 punti causali distinti e li ordina in modo crescente
    scambia i valori tra i parent tra il punto 0 e l'1 e quelli da dopo il 2
    """
    points = sorted(np.random.choice(len(parent1), size=3, replace=False))
    c1, c2 = parent1[:], parent2[:]
    c1[points[0]:points[1]], c2[points[0]:points[1]] = parent2[points[0]:points[1]], parent1[points[0]:points[1]]
    c1[points[2]:], c2[points[2]:] = parent2[points[2]:], parent1[points[2]:]
    return c1, c2

def mutate(solution, mutation_rate):
    """
    Mutazione con flipping dei bit (NumPy).
    con probabilità mutation_rate sceglie dei bit
    poi prende la soluzione e controlla ogni bit, se quel bit era stato scelto allora lo inverte
    """
    mutation_mask = np.random.rand(len(solution)) < mutation_rate
    return [(1 - bit) if mask else bit for bit, mask in zip(solution, mutation_mask)]

def local_search(solution, graph):
    """
    GREEDY
    Ricerca locale: rimuove nodi selezionati se l'aciclicità è mantenuta.
    utilizzo un massimo di iterazioni per rendere meno pesante la ricerca
    """
    max_iter = 30
    count = 0
    improved_solution = solution[:]
    for i, bit in enumerate(solution):
        if bit == 1:
            improved_solution[i] = 0
            if not is_acyclic(graph, improved_solution):
                improved_solution[i] = 1  # Ripristina se necessario
        count += 1
        if count >= max_iter:
            break
    return improved_solution

def visualize_final_graph(graph, solution):
    """
    Visualizza il grafo finale dopo aver rimosso i nodi selezionati.
    """
    # Identifica i nodi da rimuovere
    nodes_to_remove = [node for node, selected in enumerate(solution, start=1) if selected == 1]
    
    # Crea una copia del grafo e rimuovi i nodi selezionati
    final_graph = graph.copy()
    final_graph.remove_nodes_from(nodes_to_remove)
    
    # Visualizza il grafo finale
    plt.figure(figsize=(8, 6))
    pos = nx.kamada_kawai_layout(final_graph)  # Disposizione dei nodi
    nx.draw(final_graph, pos, with_labels=True, node_size=600, node_color="lightgreen", edge_color="gray", font_weight="bold")
    plt.title("Grafo finale dopo la rimozione dei nodi selezionati", fontsize=14)
    plt.show()

# ---------------------------
# Algoritmo HGA principale
# ---------------------------

def HGA(graph, node_weights, pop_size=100, generations=1001, mutation_rate=0.01):
    num_nodes = graph.number_of_nodes()
    #inizializzazione della popolazione
    population = initialize_population(pop_size, num_nodes, graph)
    
    fitnesses = [fitness(sol, node_weights, graph) for sol in population]
    best_solution, best_fitness = min(zip(population, fitnesses), key=lambda x: x[1])

    history = np.zeros((int(generations/50) + 1, int(generations/50) + 1))
    cont = 0

    #ciclo principale
    for gen in range(generations):
        new_population = []
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
 
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            child1 = local_search(child1, graph)
            child2 = local_search(child2, graph)

            new_population.extend([child1, child2])

        #aggiunta della nuova popolazione a quella vecchia
        population.extend(new_population)
        #riordinamento della popolazione
        population = sorted(population, key=lambda ind: fitness(ind, node_weights, graph))[:pop_size]
        current_fitness = fitness(population[0], node_weights, graph)
        current_best = population[0]

        #controllo fitness
        if current_fitness < best_fitness:
            best_solution, best_fitness = current_best, current_fitness

        if gen % 50 == 0:
            print(f"Generazione {gen}: Fitness migliore = {best_fitness}")
            history[0][cont] = gen
            history[1][cont] = best_fitness
            cont += 1

    visualize_final_graph(graph, best_solution)
    
    return best_solution, best_fitness, history

In [ ]:
instance_folder = "wfvs-instances-progetto-AI"

# Ottiengo tutti i file nella cartella
instance_files = [os.path.join(instance_folder, f) for f in os.listdir(instance_folder) if f.endswith(".fvs")]

for istance_file in instance_files:

    #prendo i dati dal file
    path = istance_file
    node_weights, edges = parse_fvs_file(path)

    # Costruzione del grafo
    G = nx.Graph()
    G.add_nodes_from(node_weights.keys())
    G.add_edges_from(edges)

    #stampa densità e numero componenti connesse
    density = nx.density(G)
    print("Densità del grafo:", density)
    num_components = nx.number_connected_components(G)
    print("Numero di componenti connesse:", num_components)

    # Cicli nel grafo
    try:
        cycles = list(nx.simple_cycles(G)) if G.is_directed() else list(nx.cycle_basis(G))
        print("Numero di cicli:", len(cycles))
    except:
        print("Il grafo è troppo grande per trovare tutti i cicli.")

    # Assegno i pesi come attributo dei nodi
    nx.set_node_attributes(G, node_weights, "weight")

    # Visualizzazione del grafo
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G, seed=42)
    weights = nx.get_node_attributes(G, "weight")
    nx.draw(G, pos, with_labels=False, node_size=600, node_color="skyblue", edge_color="gray", font_weight="bold")
    nx.draw_networkx_labels(G, pos, labels={n: f"{n}\n ({w})" for n, w in weights.items()}, font_size=10)
    plt.title("Visualizzazione del grafo con pesi dei nodi", fontsize=14)
    plt.show()

    evaluation_counts = []
    best_fitness_history = []

    #creazione dataframe pandas
    columns = ["istance","Fitness Run","Best Fitness mean", "Best Fitness dev", "Evaluation mean"]
    results_df = pd.DataFrame(columns=columns)

    #10 run ogni istanza
    for run in range(10):
        print("RUN: ",run+1)
        evaluation_count = 0  # Resetta il contatore
        best_solution, best_fitness, history = HGA(G, node_weights)
        evaluation_counts.append(evaluation_count)
        best_fitness_history.append(best_fitness)
        print(best_solution, best_fitness)
        print(evaluation_count, evaluation_counts)
        print(best_fitness_history)
        plt.plot(history[0], history[1], marker='o')
        plt.xlabel("Generazioni")
        plt.ylabel("Fitness migliore")
        plt.title("Convergenza dell'algoritmo")
        plt.grid(True)
        plt.show()

    #aggiunta elementi al dataframe
    new_row = pd.DataFrame({
        "istance": [path],
        "Fitness Run": [best_fitness_history],
        "Best Fitness mean": [np.mean(best_fitness_history)],
        "Best Fitness dev": [np.std(best_fitness_history)],
        "Evaluations mean": [np.mean(evaluation_counts)]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    print("media fitness: ",np.mean(best_fitness_history))
    print("deviazione fitness: ",np.std(best_fitness_history))
    print("media evaluation: ",np.mean(evaluation_counts))

    results_df.to_csv("results.xlsx", index=False, mode="a", header=not os.path.exists("results.xlsx"))